# Generating the Flextegrity Lattice

Our purpose here is to elaborate a classic Python framework for storing and writing geometric information in a variety of formats, suitable for ray tracing and use within CAD programs.

The general case CCP lattice will form our background.  Where spheres touch, they define the diamond faces of a rhombic dodecahedron.  This marks the "high water mark" for sphere size.

As spheres begin to contract, without losing their positions, we're in a position to study joining or holding strategies.  How might these spheres still connect to each other in a way that preserves their intitial positions, and yet may allow from abberrational distortions, i.e. flexibility.  Our lattice is not brittle.

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/4249887521/in/photolist-7L5wXK-bFHyv8-8bdGfA-5xKsWB-6iQiQi-aBoaMG-9f5Bg7-6iUv5o-8YLgPj-7txMuM-7vYsSk-cwtfnN-9Y6Jq5-VBwCaX-7Wkd9P-7vYsE6-6CyK5a-7YNVtZ-83biTg-7FqRR3-7NGzHS-7QW944-7wzTcq-6tQosA-5QB1z4-7YS7Uo-8pNRGs-6h9qoA-pkPux3-6CCTpC-7r71wq-9WvZsv-7nJjtA-91Zd3u-68oXAX-91W6pn-7KvFwT-7Pz6gD-8oTV56-8UWce6-7txMip-8iYwQE-crzsp1-7txPja-9Y8Bvq-zCJpe6-Hie7g4-8ryyt2-P6io19-dPU3pc" title="Flextegrity Model"><img src="https://live.staticflickr.com/2650/4249887521_9525392db4_z.jpg" width="640" height="480" alt="Flextegrity Model"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Our cubic reference cell is defined by the short diagonals of the rhombic dodecahedron's diamond faces.

Consider an icosahedron with its three XYZ golden rectangles parallel to those of our reference cube.  If the rectangles terminate in the cube's faces, then connecting bands to adjacent cube edges will provide a mechanism for inter-icosahedron joining.

As the icosahedrons grow such that their phi rectangles protrude through the cube's faces, into the neighboring cube domains, all of which are void (empty of sphere centers), the bands to the reference cube edges will slant, like the sides of a tent from a ridge pole, but not as steeply. This slanting band will meet another at 180 degrees, coming from a neighboring icosahedron.  Twelve icosahedron's around a nuclear icosahedron, will be held in place in this way.

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/3484407906/in/photolist-7L5wXK-bFHyv8-8bdGfA-5xKsWB-6iQiQi-aBoaMG-9f5Bg7-6iUv5o-8YLgPj-7txMuM-7vYsSk-cwtfnN-9Y6Jq5-VBwCaX-7Wkd9P-7vYsE6-6CyK5a-7YNVtZ-83biTg-7FqRR3-7NGzHS-7QW944-7wzTcq-6tQosA-5QB1z4-7YS7Uo-8pNRGs-6h9qoA-pkPux3-6CCTpC-7r71wq-9WvZsv-7nJjtA-91Zd3u-68oXAX-91W6pn-7KvFwT-7Pz6gD-8oTV56-8UWce6-7txMip-8iYwQE-crzsp1-7txPja-9Y8Bvq-zCJpe6-Hie7g4-8ryyt2-P6io19-dPU3pc/" title="Flextegrity Design"><img src="https://live.staticflickr.com/3539/3484407906_47eec921c5_z.jpg" width="640" height="480" alt="Flextegrity Design"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

In [1]:
class Polyhedron:
    """
    Designed to be subclassed, not used directly
    """
    
    def scale(self, scalefactor):
        newverts = {}
        for v in self.vertexes:
            newverts[v] = self.vertexes[v] * scalefactor
        newme = type(self)()
        newme.vertexes = newverts
        newme.edges = newme._distill()
        return newme

    __mul__ = __rmul__ = scale

    def translate(self, vector):
        newverts = {}
        for v in self.vertexes:
            newverts[v] = self.vertexes[v] + vector
        newme = type(self)()
        newme.vertexes = newverts
        newme.edges = newme._distill()
        return newme

    __add__ = __radd__ = translate
    
    def _distill(self):

        edges = []
        unique = set()
        
        for f in self.faces:
            for pair in zip(f , f[1:] + (f[0],)):
                unique.add( tuple(sorted(pair)) )
        
        for edge in unique:
            edges.append( Edge(self.vertexes[edge[0]],
                               self.vertexes[edge[1]]) )

        return edges            

In [2]:
class Edge:

    """
    Edges are defined by two Vectors (above) and express as cylinder via draw().
    """

    def __init__(self, v0, v1):
        self.v0 = v0
        self.v1 = v1
             
    def __repr__(self):
        return 'Edge from %s to %s' % (self.v0, self.v1)

In [3]:
def draw_vert(v): pass
def draw_face(f): pass
def draw_edge(e): pass

def draw_poly(p, the_file, v=True, f=True, e=True):
    
    ec = p.edge_color
    er = p.edge_radius
    vc = p.vert_color
    vr = p.vert_radius
    fc = p.face_color
    
    if v:
        for v in p.vertexes.values():
            draw_vert(v, vc, vr, the_file)

    if f:
        for f in p.faces:
            draw_face(f, fc, the_file)

    if e:
        for e in p.edges:
            draw_edge(e, ec, er, the_file)

In [4]:
from qrays import Qvector

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [5]:
import math
PHI = (1 + math.sqrt(5))/2.0

A = Qvector((1,0,0,0))
B = Qvector((0,1,0,0))
C = Qvector((0,0,1,0))
D = Qvector((0,0,0,1))
E,F,G,H     = B+C+D, A+C+D, A+B+D, A+B+C
I,J,K,L,M,N = A+B, A+C, A+D, B+C, B+D, C+D
O,P,Q,R,S,T = I+J, I+K, I+L, I+M, N+J, N+K
U,V,W,X,Y,Z = N+L, N+M, J+L, L+M, M+K, K+J

# OPPOSITE DIAGONALS
# ZY WX
# RV OS
# TU PQ
control = (Z - T).length()

midface = (Z + Y)
gold    = 0.5 * PHI * midface/midface.length()
Zi = gold + J/J.length() * control/2
Yi = gold + M/M.length() * control/2

midface = (W + X)
gold    = 0.5 * PHI * midface/midface.length()
Wi = gold + J/J.length() * control/2
Xi = gold + M/M.length() * control/2

midface = (R + V)
gold    = 0.5 * PHI * midface/midface.length()
Ri = gold + I/I.length() * control/2
Vi = gold + N/N.length() * control/2

midface = (O + S)
gold    = 0.5 * PHI * midface/midface.length()
Oi = gold + I/I.length() * control/2
Si = gold + N/N.length() * control/2

midface = (T + U)
gold    = 0.5 * PHI * midface/midface.length()
Ti = gold + K/K.length() * control/2
Ui = gold + L/L.length() * control/2

midface = (P + Q)
gold    = 0.5 * PHI * midface/midface.length()
Pi = gold + K/K.length() * control/2
Qi = gold + L/L.length() * control/2

In [6]:
class Tetrahedron(Polyhedron):
    
    def __init__(self):
        self.name = "Tetrahedron"
        self.edge_color = "rgb <{}, {}, {}>".format(1, 127/255, 80/255)
        self.edge_radius= 0.03
        self.vert_color = "rgb <{}, {}, {}>".format(1, 127/255, 80/255)
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"
        
        self.vertexes = dict(a = Qvector((1,0,0,0)),
                             b = Qvector((0,1,0,0)),
                             c = Qvector((0,0,1,0)),
                             d = Qvector((0,0,0,1)))
        
        # 4 faces
        self.faces = (('a','b','c'),('a','c','d'),
                      ('a','d','b'),('b','d','c'))

        self.edges = self._distill()
        
class Cube (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <0, 1, 0>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <1, 0, 0>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(a =  Qvector((1,0,0,0)),
                             b =  Qvector((0,1,0,0)),
                             c =  Qvector((0,0,1,0)),
                             d =  Qvector((0,0,0,1)),
                             e = -Qvector((1,0,0,0)),
                             f = -Qvector((0,1,0,0)),
                             g = -Qvector((0,0,1,0)),
                             h = -Qvector((0,0,0,1)))
        self.name = "Cube"
        
        # 6 faces
        self.faces = (('a','f','c','h'),('h','c','e','b'),
                      ('b','e','d','g'),('g','d','f','a'),
                      ('c','f','d','e'),('a','h','b','g'))

        self.edges = self._distill()

In [7]:
t = Tetrahedron()
t2 = t + Qvector((2,1,1,0))
t2.edges

[Edge from ivm_vector(a=2, b=1, c=2, d=0) to ivm_vector(a=1, b=0, c=0, d=0),
 Edge from ivm_vector(a=2, b=2, c=1, d=0) to ivm_vector(a=2, b=1, c=2, d=0),
 Edge from ivm_vector(a=2, b=2, c=1, d=0) to ivm_vector(a=1, b=0, c=0, d=0),
 Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=2, b=2, c=1, d=0),
 Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=1, b=0, c=0, d=0),
 Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=2, b=1, c=2, d=0)]

In [8]:
c = Cube()

In [9]:
pov_header = \
"""
// Persistence of Vision Ray Tracer Scene Description File
// File: xyz.pov
// Vers: 3.6
// Desc: test file
// Date: Sat Sep  7 09:49:33 2019
// Auth: me
// ==== Standard POV-Ray Includes ====
#include "colors.inc"     // Standard Color definitions
#include "textures.inc"   // Standard Texture definitions
#include "functions.inc"  // internal functions usable in user defined functions

// ==== Additional Includes ====
// Don't have all of the following included at once, it'll cost memory and time
// to parse!
// --- general include files ---
#include "chars.inc"      // A complete library of character objects, by Ken Maeno
#include "skies.inc"      // Ready defined sky spheres
#include "stars.inc"      // Some star fields
#include "strings.inc"    // macros for generating and manipulating text strings

// --- textures ---
#include "finish.inc"     // Some basic finishes
#include "glass.inc"      // Glass textures/interiors
#include "golds.inc"      // Gold textures
#include "metals.inc"     // Metallic pigments, finishes, and textures
#include "stones.inc"     // Binding include-file for STONES1 and STONES2
#include "stones1.inc"    // Great stone-textures created by Mike Miller
#include "stones2.inc"    // More, done by Dan Farmer and Paul Novak
#include "woodmaps.inc"   // Basic wooden colormaps
#include "woods.inc"      // Great wooden textures created by Dan Farmer and Paul Novak

global_settings {assumed_gamma 1.0}
global_settings {ambient_light rgb<1, 1, 1> }

// perspective (default) camera
camera {
  location  <4, 0.1, 0.2>
  look_at   <0.0, 0.0,  0.0>
  right     x*image_width/image_height
}

// create a regular point light source
light_source {
  0*x                  // light's position (translated below)
  color rgb <1,1,1>    // light's color
  translate <-20, 15, 10>
}

// create a regular point light source
light_source {
  0*x                  // light's position (translated below)
  color rgb <1,1,1>    // light's color
  translate <20, -15, -10>
}

background { color rgb <1.0, 1.0, 1.0> }
"""

In [10]:
def draw_vert(v, c, r, t): 
    v = v.xyz()
    x,y,z = v.x, v.y, v.z
    data = "< %s, %s, %s >" % (x,y,z), r, c
    template = ("sphere { %s, %s texture "
                "{ pigment { color %s } } no_shadow }")
    print(template % data, file=t)
    
def draw_face(f, c, t): pass

def draw_edge(e, c, r, t):
    v = e.v0.xyz()
    v0 = "< %s, %s, %s >" % (v.x, v.y, v.z)
    v = e.v1.xyz()
    v1 = "< %s, %s, %s >" % (v.x, v.y, v.z)
    data = (v0, v1, r, c)
    template = ("cylinder { %s, %s, %s texture "
                        "{pigment { color %s } } no_shadow }")
    print(template % data, file=t)

In [11]:
with open("testme.pov", "w") as target:
    target.write(pov_header)
    draw_poly(c, target, f=False)
    draw_poly(t, target, f=False)

In [12]:
from itertools import permutations
g = permutations((2,1,1,0))
unique = {p for p in g}  # set comprehension
print(unique)

{(0, 1, 1, 2), (1, 0, 1, 2), (2, 0, 1, 1), (0, 2, 1, 1), (0, 1, 2, 1), (1, 2, 1, 0), (1, 1, 2, 0), (2, 1, 1, 0), (1, 0, 2, 1), (1, 2, 0, 1), (2, 1, 0, 1), (1, 1, 0, 2)}


In [13]:
def twelve_cubes():
    c = Cube()
    twelve_cubes = [ ]
    for v in unique:
        trans_vector = Qvector(v)
        twelve_cubes.append(c + trans_vector)
    return twelve_cubes

with open("multicube.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)

![](multicube.png)

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [14]:
class Cuboctahedron (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <1, 1, 0>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <1, 1, 0>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(o =  O,
                             p =  P,
                             q =  Q,
                             r =  R,
                             s =  S,
                             t =  T,
                             u =  U,
                             v =  V,
                             w =  W,
                             x =  X,
                             y =  Y,
                             z =  Z)
        self.name = "Cuboctahedron"
        
        # 6 faces
        self.faces = (('o','w','s','z'),('z','p','y','t'),
                      ('t','v','u','s'),('w','q','x','u'),
                      ('o','p','r','q'),('r','y','v','x'),
                      ('z','s','t'),('t','y','v'),
                      ('y','p','r'),('r','q','x'),
                      ('x','u','v'),('u','s','w'),
                      ('w','q','o'),('o','z','p'))

        self.edges = self._distill()

In [28]:
with open("cubocta.pov", "w") as target:
    target.write(pov_header) 
    cubocta = Cuboctahedron()
    draw_poly(cubocta, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)

![ ](cubocta.png)

In [16]:
with open("sphere_pack.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    cub.vert_radius = Qvector((2,1,1,0)).length()/2
    cub.vert_color = "rgb <1,0,0>"
    draw_poly(cub, target, v=True, e=False, f=False)
    draw_vert(Qvector((0,0,0,0)), c=cub.vert_color, 
              r=cub.vert_radius, t=target)
    

In [27]:
# !/usr/local/bin/povray +A +H768 +W1024 sphere_pack.pov

![ ](sphere_pack.png)

In [17]:
sm_radius = (A-F).length()/2

In [18]:
with open("smaller_sphere_pack.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    cub.vert_radius = sm_radius
    cub.vert_color = "rgb <1,0,0>"
    draw_poly(cub, target, v=True, e=False, f=False)
    draw_vert(Qvector((0,0,0,0)), c=cub.vert_color, 
              r=cub.vert_radius, t=target)
    

![ ](smaller_sphere_pack.png)

In [19]:
class Icosahedron (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <0, 1, 1>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <0, 1, 1>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(o =  Oi,
                             p =  Pi,
                             q =  Qi,
                             r =  Ri,
                             s =  Si,
                             t =  Ti,
                             u =  Ui,
                             v =  Vi,
                             w =  Wi,
                             x =  Xi,
                             y =  Yi,
                             z =  Zi)
        self.name = "Icosahedron"
        
        # 20 faces
        # OPPOSITE DIAGONALS of cubocta
        # ZY WX
        # RV OS
        # TU PQ
        self.faces = (('o','w','s'),('o','z','s'),
                      ('z','p','y'),('z','t','y'),
                      ('t','v','u'),('t','s','u'),
                      ('w','q','x'),('w','u','x'),
                      ('p','o','q'),('p','r','q'),
                      ('r','y','v'),('r','x','v'),
                      ('z','s','t'),('t','y','v'),
                      ('y','p','r'),('r','q','x'),
                      ('x','u','v'),('u','s','w'),
                      ('w','q','o'),('o','z','p'))

        self.edges = self._distill()

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [20]:
ico = Icosahedron()

In [21]:
with open("jitterbug.pov", "w") as target:
    target.write(pov_header) 
    c   = Cube()
    ico = Icosahedron() * (1/PHI * math.sqrt(2)/2)
    draw_poly(c, target, f=False)
    draw_poly(ico, target, f=False)

![icosa](jitterbug.png)

In [22]:
with open("flextegrity.pov", "w") as target:
    target.write(pov_header) 
    c   = Cube()
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    ico = Icosahedron() * (1/PHI * math.sqrt(2)/2)
    draw_poly(ico, target, f=False)
    for v in unique:
        newico = ico + Qvector(v)
        draw_poly(newico, target, f=False)

![](flextegrity.png)